In [1]:
# Use autoplot 207's code for this app
import sys
from datetime import datetime

import numpy as np
from pyproj import Transformer

import geopandas as gpd
import pandas as pd
from iemweb.autoplot.scripts200.p207 import USEME, add_zeros, compute_grid_bounds, do_analysis, load_data
from matplotlib.patches import Rectangle
from pyiem.nws.vtec import NWS_COLORS
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_sqlalchemy_conn, utc
from shapely.geometry import Point

In [2]:
STORM_NUMBER = 18
WINTER = "2024-2025"
TITLE = "19-20 March 2025"
SUBTITLE = "8 AM 20 March 2025"
SETPOINT_LOCS = {}
# naive US Central local time
sts = datetime(2025, 3, 19, 12)
ets = datetime(2025, 3, 20, 19)
# Get available data
ctx = {
    "coop": "yes",
    "cocorahs": "yes",
    "t": "state",
    "sz": 30,
    "z": "yes",
    "f": "linear",
    "v": "snow",
    "wfo": "DMX",
}
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, "IA")
df = load_data(ctx, sts, ets)
df = df[~df["nwsli"].isin(["DSXI4", "DMX"])]
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
            """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
            conn,
            params=(sts - datetime.timedelta(days=1), ets),
        )
    print(df[df["state"] == "IA"])
    mp.plot_values(
        df.lon.values,
        df.lat.values,
        df.magnitude.values,
        fmt="%.2f",
        labelbuffer=1,
        color="purple",
    )


def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE,),
        subtitle=(
            f"Snowfall totals till {SUBTITLE} from NWS COOP, LSR, CoCoRaHS Reports; "
            f"IEM {WINTER} Winter Storm #{STORM_NUMBER}"
        ),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2["val"] >= lower) & (df2["val"] < upper)]
    print(df[df["state"] == "IA"])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(
            df_useme_plot["lon"],
            df_useme_plot["lat"],
            df_useme_plot["val"].values,
            "%s",
            labels=df_useme_plot["nwsli"].values,
            textsize=10,
            labeltextsize=10,
            labelbuffer=1,
        )
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(
            SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1]
        )
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True


def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.0
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz / 2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz / 2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1


def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets),
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0,), ("Snow Squall Warning",), loc=1).set_zorder(
        1000
    )

In [5]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,WY,RIW,20.0,-109.240000,44.300000,POINT (-733367.207 -36058.77),True,0,True,LSR,-33,16
1,WY,RIW,18.3,-110.670000,44.130000,POINT (-848562.474 -40919.274),True,1,True,LSR,-36,16
2,WY,RIW,18.0,-110.950000,43.790000,POINT (-875695.808 -75506.513),True,2,True,LSR,-37,15
3,WY,RIW,16.0,-109.910000,44.730000,POINT (-780421.761 17764.169),True,3,True,LSR,-34,18
4,WY,RIW,16.0,-109.790000,44.380000,POINT (-775698.724 -22092.707),True,4,True,LSR,-34,17
...,...,...,...,...,...,...,...,...,...,...,...,...
869,Z,NaN,0.0,-94.833132,39.992809,POINT (440227.691 -542858.648),True,Z1,False,NaN,6,0
870,Z,NaN,0.0,-90.630535,39.708707,POINT (800227.691 -542858.648),True,Z2,False,NaN,18,0
871,Z,NaN,0.0,-94.769122,40.800868,POINT (440227.691 -452858.648),True,Z3,False,NaN,6,3
872,Z,NaN,0.0,-93.702848,40.744334,POINT (530227.691 -452858.648),True,Z4,False,NaN,9,3


In [17]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = [
        'NE-WS-13',
        'ILNI2',
        384,
        'SACI4',
        368,
        249,
        'SANI4',
    ]
    if cull:
        df.loc[df["nwsli"].isin(cull), USEME] = False
    hardcode = [
    ]
    for nwsli, val in hardcode:
        df.loc[df["nwsli"] == nwsli, "val"] = val

    ctx["csector"] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=3, upper=5)
    #draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename="250320.png")
    mp.close()


main()

    state  wfo      val        lon        lat                             geo  \
7      IA  DMX  14.5000 -94.090000  43.380000  POINT (477161.787 -162835.961)   
14     IA  DMX  12.5000 -94.240000  43.050000   POINT (467622.741 -200326.38)   
17     IA  DMX  12.0000 -94.160000  43.420000  POINT (471208.541 -158804.024)   
23     IA  DMX  11.5000 -94.230000  43.070000  POINT (468278.644 -198049.487)   
25     IA  DMX  11.0000 -93.940000  43.390000  POINT (489163.688 -160837.545)   
..    ...  ...      ...        ...        ...                             ...   
854    IA  OAX   0.0001 -95.698426  40.724626  POINT (362540.206 -465841.464)   
855    IA  DVN   0.0001 -92.084746  41.762082  POINT (655507.897 -328377.824)   
857    IA  DVN   0.0001 -90.206599  41.861460   POINT (808940.48 -300612.701)   
858    IA  DMX   0.0001 -93.719400  41.787000  POINT (520298.314 -337320.838)   
860    IA  DVN   0.0001 -91.489084  41.803983  POINT (704157.816 -318802.654)   

     used_for_analysis     

## 